In [1]:
# Install packages (one-time)

In [2]:
%%bash
source $VIRTUAL_ENV_DIR/python3/bin/activate

install_package_python3.sh add dsw_qr==0.1.13

$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo-py
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install tchannel


Updating dependencies
Resolving dependencies...


Package operations: 1 install, 52 updates, 0 removals

  - Updating six (1.14.0 -> 1.15.0)
  - Updating tornado (4.5.3 -> 5.1.1)
  - Updating colorama (0.4.3 -> 0.4.4)
  - Updating packaging (20.3 -> 20.4)
  - Updating jinja2 (2.11.1 -> 2.11.2)
  - Updating cachetools (3.1.1 -> 4.1.1)
  - Updating certifi (2020.4.5.1 -> 2020.6.20)
  - Updating cffi (1.14.0 -> 1.14.3)
  - Updating idna (2.9 -> 2.10)
  - Updating numpy (1.16.4 -> 1.16.6)
  - Updating smmap (3.0.1 -> 3.0.4)
  - Updating urllib3 (1.25.8 -> 1.25.11)
  - Updating bcrypt (3.1.7 -> 3.2.0)
  - Updating clay-config-file (1.2.0 -> 1.2.1)
  - Updating click (7.1.1 -> 7.1.2)
  - Updating cryptography (2.9 -> 3.2)
  - Updating gitdb (4.0.2 -> 4.0.5)
  - Updating grpcio (1.28.1 -> 1.29.0)
  - Updating psutil (5.7.0 -> 5.7.3)
  - Updating py4j (0.10.4 -> 0.10.7)
  - Updating pyarrow (0.16.0 -> 2.0.0)
  - Updating pynacl (1.3.0 -> 1.4.0)
  - Updating requests (2.23.0 -> 2.24.0)
  - Upd

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 0.71.0 requires cachetools>=4.0, but you have cachetools 3.1.1 which is incompatible.
streamlit 0.71.0 requires tornado>=5.0, but you have tornado 4.5.3 which is incompatible.
mxpkg 1.1.49 requires bcrypt==3.1.7, but you have bcrypt 3.2.0 which is incompatible.
mxpkg 1.1.49 requires certifi==2020.4.5.1, but you have certifi 2020.6.20 which is incompatible.
mxpkg 1.1.49 requires cffi==1.14.0, but you have cffi 1.14.3 which is incompatible.
mxpkg 1.1.49 requires clay-config-file==1.2.0, but you have clay-config-file 1.2.1 which is incompatible.
mxpkg 1.1.49 requires click==7.1.1, but you have click 7.1.2 which is incompatible.
mxpkg 1.1.49 requires colorama==0.4.3, but you have colorama 0.4.4 which is incompatible.
mxpkg 1.1.49 requires cryptography==2.9, but you have cryptography 3.2 which is incompatible

In [3]:
# fetch data

In [4]:
import os

import pandas as pd
from dsw_qr import dsw_qr

2022-08-08 18:54:50,623 jaeger_tracing WARNING Jaeger tracer already initialized, skipping


In [5]:
USER_EMAIL = 'targupt@uber.com'

In [6]:
QUERY = """select
  derived_eta,
  derived_surge_diff,
  derived_offered_demand_surge_avg_4week,
  derived_offered_demand_surge_avg_1week,
  derived_offered_forward_dispatched_avg_4week,
  derived_offered_forward_dispatched_avg_1week,
  derived_offered_trip_distance_haversine_avg_4week,
  derived_offered_trip_distance_haversine_avg_1week,
  derived_days_since_signup,
  derived_days_since_first_trip,
  calibrated_prob
from
  intelligent_dispatch.targupt_driver_cancel_premium_v1
where
  mega_region = 'EMEA'
  and country_id not in (184, 125)
  and abs(mod(hash(job_uuid), 100)) < 2"""

In [9]:
from queryrunner_client import Client
qr = Client(user_email=USER_EMAIL)
qr.list_datasources()
cursor = qr.execute("hive", QUERY)
result = cursor.fetchall()

08/08/2022 06:55:44 PM Send empty tier_metadata {} to queryrunner. Query is default to tier 5.
08/08/2022 06:55:44 PM  [Polling] 0fb9553f-b65f-469d-8710-44cd33133378 
08/08/2022 06:55:44 PM  [Status] created 
08/08/2022 06:55:45 PM  [Status] started waiting to execute 
08/08/2022 06:55:46 PM  [Status] started execution 
2022-08-08 18:56:28,598 galileo.interceptors.http WARNING Destination name not provided; HTTP request needs to contain destination service name
2022-08-08 18:56:28,599 galileo.interceptors.http WARNING Failed to obtain service name from port mappings; outbound request will not contain auth tokens
2022-08-08 18:57:28,590 galileo.interceptors.http WARNING Destination name not provided; HTTP request needs to contain destination service name
2022-08-08 18:57:28,591 galileo.interceptors.http WARNING Failed to obtain service name from port mappings; outbound request will not contain auth tokens
2022-08-08 18:58:28,598 galileo.interceptors.http WARNING Destination name not pro

In [10]:
pd.DataFrame(result).to_csv("data/emea_premium_driver_prediction_sample.csv", index=False)